In [ ]:
import os
from model import *
conversation = get_conversation()

In [ ]:
import re
def extracter(text):
    input_match = re.search(r'input:\s*(.*?)\n', text, re.IGNORECASE | re.DOTALL)
    output_match = re.search(r'output:\s*(.*?)\n', text, re.IGNORECASE | re.DOTALL)
    instruction_match = re.search(r'instruction:\s*(.*)', text, re.IGNORECASE | re.DOTALL)
    
    # Bölümleri çıkarıyoruz
    input_text = input_match.group(1).strip() if input_match else None
    output_text = output_match.group(1).strip() if output_match else None
    instruction_text = instruction_match.group(1).strip() if instruction_match else None
    
    return input_text, output_text, instruction_text

In [ ]:
import pandas as pd
import json

def parquet_to_json(parquet_file, json_file):
    df = pd.read_parquet(parquet_file)
    json_str = df.to_json(orient='records', lines=False)
    with open(json_file, 'w') as f:
        f.write(json_str)

    
def json_lang_convert(original_file_path):

    # Öncü json dosyası oluşturma işlemi
    with open(original_file_path, 'r') as file:
        original = json.load(file)
    
    features = original[0]
    
    with open('TR.json', 'w') as file:
        json.dump({'0': features}, file, indent=4)

    # Türkçe dönüşüm döngüsü:
    with open('TR.json', 'r') as file:
        new_json = json.load(file)

    hata_kaydedicisi = 0

    while True:
        try:
            for i in range(hata_kaydedicisi, len(original)+1):
                hata_kaydedicisi = i
                
                last_index = str(len(new_json))
                #new_key = str(i)
        
                entry = original[i]
                combined_text = f"Input: {entry['input']}\nOutput: {entry['output']}\nInstruction: {entry['instruction']}"
                
                response = conversation.predict(human_input= combined_text)
                
                input, output, instruction = extracter(response)
                
                if not (input and output and instruction):
                    print(f" En az bir output boş geldi->")
                    print("Tekrar deneniyor...")
                    print("input:",input,"\nOutput:",output,"\ninstruction",instruction)
                    print(i/0)
                    
                new_element = {
                    "input": input,
                    "output": output,
                    "instruction": instruction
                }
                
                new_json[last_index] = new_element
                
                with open('TR.json', 'w') as file:
                    json.dump(new_json, file, indent=4)
                    
                if i % 100 == 0:
                    print(i,". satır da yazıldı")
            break
        
        except Exception as e:
            print(f"Komutan logar.. Bir hata yaklaşıyor şu sayfada -> {hata_kaydedicisi}: {e}")
            print("Tekrar deneniyor...")
                

file_path = 'train-00000-of-00001-ab79bf9300210e98.parquet'

if file_path.split('.')[-1] == 'parquet':
    parquet_to_json(file_path, "temp_parquet.json")
    json_lang_convert("./temp_parquet.json")

if file_path.split('.')[-1] == 'json':
    json_lang_convert(file_path)
